# Schema

![](https://github.com/motional/nuplan-devkit/raw/master/docs/nuplan_schema.png)

## set environment variables

In [1]:
import os
NUPLAN_DATA_ROOT = os.getenv('NUPLAN_DATA_ROOT', '/home/gac/nuplan_explore/nuplan')
NUPLAN_MAPS_ROOT = os.getenv('NUPLAN_MAPS_ROOT', '/home/gac/nuplan_explore/nuplan/dataset/maps')
NUPLAN_DB_FILES = os.getenv('NUPLAN_DB_FILES', '/home/gac/nuplan_explore/nuplan/dataset/nuplan-v1.1/mini')
NUPLAN_MAP_VERSION = os.getenv('NUPLAN_MAP_VERSION', 'nuplan-maps-v1.0')

# ORM Database API

## load data

In [ ]:
from nuplan.database.nuplan_db_orm.nuplandb_wrapper import NuPlanDBWrapper

nuplandb_wrapper = NuPlanDBWrapper(
    data_root=NUPLAN_DATA_ROOT,
    map_root=NUPLAN_MAPS_ROOT,
    db_files=NUPLAN_DB_FILES,
    map_version=NUPLAN_MAP_VERSION,
)

## query a database

In [ ]:
log_db_name = "2021.05.12.22.00.38_veh-35_01008_01518"
log_db = nuplandb_wrapper.get_log_db(log_db_name)

Get all file name

In [ ]:
from os import listdir
from os.path import isfile, join
file_names_with_extension = [f for f in listdir(NUPLAN_DB_FILES) if isfile(join(NUPLAN_DB_FILES, f))]

file_names = [name[:-3] for name in file_names_with_extension]
file_names

## acess data

In [ ]:
len(log_db.scenario_tag)

In [ ]:
len(log_db.lidar_pc)

In [ ]:
log_db.lidar_pc[0]

In [ ]:
raw_scenario = [log_db.scenario_tag[i].type for i in range(len(log_db.scenario_tag))]
set(raw_scenario)

In [ ]:
log_db.lidar_box

## filter

### import data type

In [ ]:
from nuplan.database.nuplan_db_orm.lidar_pc import LidarPc
from nuplan.database.nuplan_db_orm.category import Category
from nuplan.database.nuplan_db_orm.ego_pose import EgoPose
from nuplan.database.nuplan_db_orm.lidar_box import LidarBox
from nuplan.database.nuplan_db_orm.log import Log
from nuplan.database.nuplan_db_orm.scene import Scene
from nuplan.database.nuplan_db_orm.scenario_tag import ScenarioTag
from nuplan.database.nuplan_db_orm.traffic_light_status import TrafficLightStatus
from nuplan.database.nuplan_db_orm.track import Track

### query and filter

In [ ]:
scenario_tag_single = log_db.session.query(ScenarioTag) \
  .filter(ScenarioTag.agent_track_token != None) \
  .limit(10) \
  .all()

scenario_tag_single

In [ ]:
lidar_pc_token = scenario_tag_single[0].lidar_pc_token
agent_track_token = scenario_tag_single[0].agent_track_token

In [ ]:
lidar_pc = log_db.session.query(LidarPc) \
  .filter(LidarPc.token == lidar_pc_token) \
  .all()
          
lidar_pc

In [ ]:
scene = log_db.session.query(Scene) \
  .filter(Scene.token == '165060762e765a5a') \
  .all()
          
scene

In [ ]:
prev_token = lidar_pc[0].prev_token
lidar_pc = log_db.session.query(LidarPc) \
  .filter(LidarPc.token == prev_token) \
  .all()
          
lidar_pc

# Direct SQL API

In [ ]:
from nuplan.database.nuplan_db.query_session import execute_one, execute_many

In [ ]:
query = """
SELECT COUNT(*) AS cnt
FROM lidar_pc;
"""

result = execute_one(query, (), os.path.join(NUPLAN_DB_FILES, f"{log_db_name}.db"))
print(f"The number of lidar_pcs in this log files is {result['cnt']}.")

In [ ]:
example_token = "e1e4ee25d1ff58f2"
query = """
SELECT ep.x AS ep_x,
       ep.y AS ep_y,
       ep.z AS ep_z,
       lp.token AS token
FROM ego_pose AS ep
INNER JOIN lidar_pc AS lp
  ON lp.ego_pose_token = ep.token
WHERE lp.token = ?
"""

result = execute_one(
    query, 
    (bytearray.fromhex(example_token),), 
    os.path.join(NUPLAN_DB_FILES, f"{log_db_name}.db")
)

print(f"Lidar_pc token: {result['token'].hex()}.")
print(f"Ego pose: <{result['ep_x']}, {result['ep_y']}, {result['ep_z']}>.")

# NuPlanScenarios

### import

In [2]:
import itertools
import logging
import random
from collections import defaultdict
from dataclasses import dataclass
from os.path import join
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import numpy.typing as npt

from nuplan.common.actor_state.vehicle_parameters import get_pacifica_parameters
from nuplan.common.maps.nuplan_map.map_factory import NuPlanMapFactory, get_maps_db
from nuplan.database.nuplan_db.nuplan_scenario_queries import (
    get_lidarpc_token_map_name_from_db,
    get_lidarpc_token_timestamp_from_db,
    get_lidarpc_tokens_with_scenario_tag_from_db,
)
from nuplan.planning.nuboard.base.data_class import NuBoardFile, SimulationScenarioKey
from nuplan.planning.nuboard.base.experiment_file_data import ExperimentFileData
from nuplan.planning.nuboard.base.simulation_tile import SimulationTile
from nuplan.planning.scenario_builder.abstract_scenario import AbstractScenario
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario import NuPlanScenario
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_filter_utils import discover_log_dbs
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import (
    DEFAULT_SCENARIO_NAME,
    ScenarioExtractionInfo,
)

from tutorials.utils.tutorial_utils import get_scenario_type_token_map, get_default_scenario_from_token

In [3]:
data_root=NUPLAN_DATA_ROOT
map_root=NUPLAN_MAPS_ROOT
db_files=NUPLAN_DB_FILES
map_version=NUPLAN_MAP_VERSION

### Using `NuPlanScenario` class.
Query a desired scenario via `NuPlanScenario` class, e.g., 'near_high_speed_vehicle'.

In [4]:
query_scenario = 'near_high_speed_vehicle'

log_db_files = discover_log_dbs(db_files)
scenario_type_token_map = get_scenario_type_token_map(log_db_files)

In [5]:
log_db_file, token = random.choice(scenario_type_token_map[query_scenario])
scenario = get_default_scenario_from_token(data_root, log_db_file, token, map_root, map_version)

In [6]:
len(scenario_type_token_map)

67

API params descriptions in `abstract_scenario.py`

### duration of the scenario

In [7]:
scenario.duration_s.time_s

9.400468

### number of frame the scenario contains

In [8]:
scenario.get_number_of_iterations()

95

### frame interval

In [9]:
scenario.database_interval # s

0.1

### initial ego state

location

In [ ]:
scenario.initial_ego_state.agent.center

velocity

In [ ]:
scenario.initial_ego_state.agent.velocity

acceleration

In [ ]:
scenario.initial_ego_state.dynamic_car_state.center_acceleration_2d

In [ ]:
scenario.get_ego_state_at_iteration(iteration=1).agent.center

In [ ]:
scenario.get_ego_state_at_iteration(iteration=1).agent.velocity

In [ ]:
scenario.get_ego_state_at_iteration(iteration=1).dynamic_car_state.center_acceleration_2d

In [ ]:
ego_current_state = scenario.get_ego_state_at_iteration(iteration=1)

ego_current_array = np.array(
    [ego_current_state.agent.center.x,
     ego_current_state.agent.center.y,
     ego_current_state.agent.center.heading,
     ego_current_state.dynamic_car_state.center_velocity_2d.x,
     ego_current_state.dynamic_car_state.center_velocity_2d.y,
     ego_current_state.dynamic_car_state.center_acceleration_2d.x,
     ego_current_state.dynamic_car_state.center_acceleration_2d.y]
)


### ego past trajectory

In [ ]:
ego_past_trajectory = scenario.get_ego_past_trajectory(
    iteration=0,
    time_horizon=5,
)

ego_past_trajectory

In [ ]:
past_trajectory = list(ego_past_trajectory)
past_trajectory

In [ ]:
past_trajectory[2].agent.center

In [ ]:
past_trajectory[2].dynamic_car_state.center_velocity_2d

In [ ]:
past_trajectory[2].dynamic_car_state.center_acceleration_2d

In [ ]:
ego_past_array = np.array(
    [[past_state.agent.center.x,
      past_state.agent.center.y,
      past_state.agent.center.heading,
      past_state.dynamic_car_state.center_velocity_2d.x,
      past_state.dynamic_car_state.center_velocity_2d.y,
      past_state.dynamic_car_state.center_acceleration_2d.x,
      past_state.dynamic_car_state.center_acceleration_2d.y] for past_state in past_trajectory]
)

ego_past_array.shape

### ego future trajectory

In [ ]:
ego_future_trajectory = scenario.get_ego_future_trajectory(
    iteration=0, # iteration within scenario 0 <= scenario_iteration < get_number_of_iterations
    time_horizon=10, # the desired horizon to the future, s
)

ego_future_trajectory

In [ ]:
future_trajectory = list(ego_future_trajectory)
future_trajectory

center

In [ ]:
future_trajectory[0].agent.center

velocity

In [ ]:
future_trajectory[0].dynamic_car_state.center_velocity_2d

acceleration

In [ ]:
future_trajectory[0].dynamic_car_state.center_acceleration_2d

In [ ]:
future_trajectory[0].time_seconds

In [ ]:
future_trajectory[-1].time_seconds

In [ ]:
ego_future_array = np.array(
    [[future_state.agent.center.x,
      future_state.agent.center.y,
      future_state.agent.center.heading,
      future_state.dynamic_car_state.center_velocity_2d.x,
      future_state.dynamic_car_state.center_velocity_2d.y,
      future_state.dynamic_car_state.center_acceleration_2d.x,
      future_state.dynamic_car_state.center_acceleration_2d.y] for future_state in future_trajectory]
)

ego_future_array

In [ ]:
np.linalg.norm(np.asarray([ego_future_array[-1][3], ego_future_array[-1][4]]))

### objects within time window

In [ ]:
tracked_objs_within_t = scenario.get_tracked_objects_within_time_window_at_iteration(
    iteration=0,
    past_time_horizon=5,
    future_time_horizon=0
)

tracked_objs_within_t

In [ ]:
tracked_objs_within_t.tracked_objects.get_static_objects()

In [ ]:
set([obj.metadata.category_name for obj in tracked_objs_within_t.tracked_objects.get_static_objects()])

In [ ]:
tracked_objs_within_t.tracked_objects.get_static_objects()[2].center

In [ ]:
tracked_objs_within_t.tracked_objects.get_agents()

location

In [ ]:
set([obj.metadata.category_name for obj in tracked_objs_within_t.tracked_objects.get_agents()])

In [ ]:
tracked_objs_within_t.tracked_objects.get_agents()[0].center

velocity

In [ ]:
tracked_objs_within_t.tracked_objects.get_agents()[0].velocity.x

**acceleration is NOT given in tracking**

In [ ]:
obj = tracked_objs_within_t.tracked_objects.get_agents()[3]

In [ ]:
obj.metadata

### agent at time point

In [ ]:
object_current_state = scenario.get_tracked_objects_at_iteration(iteration=1).tracked_objects.get_agents()

In [ ]:
obj = scenario.get_tracked_objects_at_iteration(iteration=1).tracked_objects.get_agents()[0]

In [ ]:
obj.velocity

In [ ]:
obj.track_token

In [ ]:
object_current_array = np.array(
    [[obj_state.center.x,
      obj_state.center.y,
      obj_state.center.heading,
      obj_state.velocity.x,
      obj_state.velocity.y,
      0,
      0] for obj_state in object_current_state]
)
object_current_track_token = np.array(
    [obj_state.track_token for obj_state in object_current_state]
)

In [ ]:
object_current_array

In [ ]:
object_current_track_token

In [ ]:
ego_x = ego_current_array[0]
ego_y = ego_current_array[1]

objects_current_info = []
objects_distance = []
objects_track_token = []
for obj_current_state in object_current_state:
    obj_x = obj_current_state.center.x
    obj_y = obj_current_state.center.y
    dx = obj_x - ego_x
    dy = obj_y - ego_y
    distance = np.linalg.norm(np.asarray([dx, dy]))

    if distance <= 1000:
        objects_current_info.append(
            [obj_current_state.center.x,
             obj_current_state.center.y,
             obj_current_state.center.heading,
             obj_current_state.velocity.x,
             obj_current_state.velocity.y,
             0,
             0,
            distance]
        )
        objects_distance.append(distance)
        objects_track_token.append(obj_current_state.track_token)
objects_current_info = np.asarray(objects_current_info)
objects_distance = np.asarray(objects_distance)
objects_track_token = np.asarray(objects_track_token)

In [ ]:
objects_current_info

In [ ]:
objects_distance

In [ ]:
objects_track_token

In [ ]:
objects_current_info[objects_distance.argsort()]

In [ ]:
objects_track_token[objects_distance.argsort()]

In [ ]:
objects_distance[objects_distance.argsort()]

In [ ]:
len(objects_current_info)

In [ ]:
objects_current_info.shape

In [ ]:
obj_token = objects_track_token[4]
obj_token

In [ ]:
obj_token = '6aef1d15a12f53bc'
np.where(objects_track_token==obj_token)[0].item() if len(np.where(objects_track_token==obj_token)[0]) != 0 else None

In [ ]:
objects_virtual_info = np.zeros((3, objects_current_info.shape[1]))
objects_virtual_info

In [ ]:
np.concatenate((objects_current_info, objects_virtual_info), axis=0)

In [ ]:
log_db_files = discover_log_dbs(db_files)
scenario_type_token_map = get_scenario_type_token_map(log_db_files)
scenario_type_list = sorted(scenario_type_token_map.keys())

In [ ]:
scenario_dict = {}
for scenario_type in scenario_type_list:
    scenario_dict[scenario_type] = len(scenario_type_token_map[scenario_type])

In [ ]:
scenario_dict

In [ ]:
sum(list(scenario_dict.values()))

In [ ]:
labels = list(scenario_dict.keys())
sizes = list(scenario_dict.values())

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(aspect="equal"))

data = sizes
ingredients = labels


def func(pct, allvals):
    absolute = int(np.round(pct/100.*np.sum(allvals)))
    return f"{pct:.1f}%\n({absolute:d})"


wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                  textprops=dict(color="w"))

ax.legend(wedges, ingredients,
          title="Scenario_tag",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))

plt.setp(autotexts, size=8, weight="bold")

ax.set_title("Scenario distribution")

plt.show()

In [14]:
agent = {
    'a1' : {'x':1, 'y':1, 'vx':1, 'vy':1, 'ax':1, 'ay':2},
    'a2' : {'x':1, 'y':1, 'vx':1, 'vy':1, 'ax':1, 'ay':3},
    'a3' : {'x':1, 'y':1, 'vx':1, 'vy':1, 'ax':1, 'ay':4}
}

In [16]:
for a in agent:
    print(agent[a]['ay'])

2
3
4


In [19]:
list(range(3, 10))

[3, 4, 5, 6, 7, 8, 9]

In [20]:
float('192944')

192944.0

In [53]:
x = np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
x

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [55]:
x = np.array([])
y = x.copy()
y.resize((6,3))
y

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])